In [2]:
# Notebook 04: Validación y Métricas de Rendimiento
# **Universidad:** Universidad Nacional Experimental de Guayana (UNEG)  
# **Asignatura:** Sistemas de Bases de Datos II  
# **Proyecto:** Proyecto N2 - Data Pipeline Escalable
# ---
# **Descripción:**  
# Valida las métricas de rendimiento y ejecuta consultas analíticas en ClickHouse.
# Este notebook recopila **automáticamente** las métricas de los notebooks anteriores.
# **Prerrequisitos:**
# - Ejecutar `02_generador_datos.ipynb`
# - Ejecutar `03_etl_spark.ipynb`
# - Los notebooks anteriores guardan sus tiempos en `../docs/metricas.json`

In [3]:
import time
import json
import os
from datetime import datetime

# Archivo de métricas compartido (directorio docs montado por Docker)
METRICS_FILE = '../docs/metricas.json'

print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📅 Fecha de ejecución: 2026-02-09 21:33:52


In [4]:
# =====================================================
# 📥 CARGAR MÉTRICAS AUTOMÁTICAMENTE
# =====================================================

if os.path.exists(METRICS_FILE):
    with open(METRICS_FILE, 'r') as f:
        metricas = json.load(f)
    print("✅ Métricas cargadas automáticamente desde notebooks anteriores")
    print(f"\n📋 Contenido de {METRICS_FILE}:")
    print(json.dumps(metricas, indent=2))
else:
    print("⚠️ No se encontró el archivo de métricas.")
    print("   Ejecuta primero los notebooks 02 y 03.")
    metricas = {}

# Extraer valores para usar en el resumen
tiempo_ingesta = metricas.get('ingesta_cassandra', {}).get('tiempo_segundos', 0)
tiempo_etl = metricas.get('etl_spark', {}).get('tiempo_total', 0)

print(f"\n🕐 Tiempos extraídos:")
print(f"   - Ingesta Cassandra: {tiempo_ingesta:.2f} seg")
print(f"   - ETL Spark Total:   {tiempo_etl:.2f} seg")

✅ Métricas cargadas automáticamente desde notebooks anteriores

📋 Contenido de ../docs/metricas.json:
{
  "ingesta_cassandra": {
    "tiempo_segundos": 13.17,
    "registros": 100000,
    "tasa_regs_seg": 7590.24,
    "timestamp": "2026-02-09T21:32:58.089691"
  },
  "etl_spark": {
    "tiempo_lectura": 0.34,
    "tiempo_transformacion": 0.89,
    "tiempo_escritura": 0.86,
    "tiempo_total": 2.09,
    "registros_entrada": 100000,
    "registros_salida": 5490,
    "timestamp": "2026-02-09T21:33:40.710514"
  },
  "consultas_clickhouse": {
    "consulta_top10": 0.0128,
    "consulta_promedio": 0.0096,
    "registros_analizados": 5651,
    "timestamp": "2026-02-07T16:32:24.060014"
  }
}

🕐 Tiempos extraídos:
   - Ingesta Cassandra: 13.17 seg
   - ETL Spark Total:   2.09 seg


In [5]:
# Instalar cliente si no existe
!pip install -q clickhouse-connect

import clickhouse_connect

# Conectar a ClickHouse
client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,
    database='dw_analitico'
)

# Verificar conexión
result = client.query("SELECT COUNT(*) as total FROM ventas_resumen")
total_registros = result.result_rows[0][0]
print(f"✅ Conectado a ClickHouse")
print(f"   - Registros en ventas_resumen: {total_registros:,}")

✅ Conectado a ClickHouse
   - Registros en ventas_resumen: 7,980


In [6]:
query_top10 = """
SELECT 
    categoria,
    sum(ventas_totales) as total_ventas_periodo
FROM dw_analitico.ventas_resumen
GROUP BY categoria
ORDER BY total_ventas_periodo DESC
LIMIT 10
"""

print("--- Ejecutando Consulta Top 10 ---")
start_time = time.time()

result_top10 = client.query(query_top10)

end_time = time.time()
tiempo_consulta_top10 = end_time - start_time

print(f"⏱️ Tiempo de ejecución: {tiempo_consulta_top10:.4f} segundos")
print("\n📊 Resultados:")
print("-" * 40)
for row in result_top10.result_rows:
    print(f"  {row[0]}: ${row[1]:,.2f}")

--- Ejecutando Consulta Top 10 ---
⏱️ Tiempo de ejecución: 0.0278 segundos

📊 Resultados:
----------------------------------------
  Mascotas: $4,244,727.13
  Jardin: $4,241,456.22
  Libros: $4,188,035.86
  Belleza: $4,168,598.34
  Videojuegos: $4,165,744.96
  Automotriz: $4,161,815.88
  Ropa: $4,149,082.77
  Deportes: $4,146,430.15
  Electronica: $4,139,323.63
  Salud: $4,113,339.42


In [7]:
query_promedio = """
SELECT 
    categoria,
    avg(ventas_totales) as promedio_ventas_diarias,
    sum(cantidad_transacciones) as transacciones_totales
FROM dw_analitico.ventas_resumen
GROUP BY categoria
ORDER BY categoria
"""

print("--- Ejecutando Consulta Promedio Diario ---")
start_time = time.time()

result_promedio = client.query(query_promedio)

end_time = time.time()
tiempo_consulta_promedio = end_time - start_time

print(f"⏱️ Tiempo de ejecución: {tiempo_consulta_promedio:.4f} segundos")
print("\n📊 Resultados:")
print("-" * 60)
for row in result_promedio.result_rows:
    print(f"  {row[0]}: Promedio ${row[1]:,.2f} | Transacciones: {row[2]:,}")

--- Ejecutando Consulta Promedio Diario ---
⏱️ Tiempo de ejecución: 0.0079 segundos

📊 Resultados:
------------------------------------------------------------
  Alimentos: Promedio $7,487.93 | Transacciones: 9,378
  Automotriz: Promedio $7,822.96 | Transacciones: 9,863
  Belleza: Promedio $7,895.07 | Transacciones: 9,744
  Cine: Promedio $7,691.39 | Transacciones: 9,552
  Deportes: Promedio $7,823.45 | Transacciones: 9,712
  Electronica: Promedio $7,737.05 | Transacciones: 9,955
  Hogar: Promedio $7,686.87 | Transacciones: 9,624
  Jardin: Promedio $7,972.66 | Transacciones: 9,820
  Juguetes: Promedio $7,678.36 | Transacciones: 9,611
  Libros: Promedio $7,828.10 | Transacciones: 9,694
  Mascotas: Promedio $8,039.26 | Transacciones: 9,785
  Musica: Promedio $7,639.86 | Transacciones: 9,478
  Ropa: Promedio $7,769.82 | Transacciones: 9,719
  Salud: Promedio $7,746.40 | Transacciones: 9,648
  Videojuegos: Promedio $7,771.91 | Transacciones: 9,770


## 6. 📋 RESUMEN FINAL DE MÉTRICAS (PARA EL INFORME)

In [8]:
import platform

print("="*70)
print("📊 RESUMEN DE MÉTRICAS DE RENDIMIENTO - PROYECTO DATA PIPELINE")
print("="*70)
print(f"\n🖥️ Entorno de Ejecución:")
print(f"   - Plataforma: Docker Container")
print(f"   - Python: {platform.python_version()}")
print(f"   - Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Objetivos
obj_ingesta = 300  # 5 minutos
obj_etl = 120  # 2 minutos
obj_query = 3  # 3 segundos

print("\n" + "-"*70)
print("\n📈 TABLA DE TIEMPOS (Copiar al informe):")
print("\n| Operación                  | Tiempo Real | Objetivo    | Cumple |")
print("|:---------------------------|:-----------:|:-----------:|:------:|")

# Ingesta
cumple_ing = "✅" if tiempo_ingesta < obj_ingesta else "❌"
print(f"| Ingesta Cassandra (100k)   | {tiempo_ingesta:>8.2f} s  | < 5 min     | {cumple_ing}     |")

# ETL
cumple_etl = "✅" if tiempo_etl < obj_etl else "❌"
print(f"| Transformación Spark ETL   | {tiempo_etl:>8.2f} s  | < 2 min     | {cumple_etl}     |")

# Consulta Top 10
cumple_q1 = "✅" if tiempo_consulta_top10 < obj_query else "❌"
print(f"| Consulta Top 10 (CH)       | {tiempo_consulta_top10:>8.4f} s  | < 3 seg     | {cumple_q1}     |")

# Consulta Promedio
cumple_q2 = "✅" if tiempo_consulta_promedio < obj_query else "❌"
print(f"| Consulta Promedio (CH)     | {tiempo_consulta_promedio:>8.4f} s  | < 3 seg     | {cumple_q2}     |")

print("\n" + "="*70)
print("💡 CONCLUSIÓN:")
todos_cumplen = all([
    tiempo_ingesta < obj_ingesta, 
    tiempo_etl < obj_etl, 
    tiempo_consulta_top10 < obj_query, 
    tiempo_consulta_promedio < obj_query
])
if todos_cumplen:
    print("   ✅ TODOS los objetivos de rendimiento fueron CUMPLIDOS.")
else:
    print("   ⚠️ Algunos objetivos NO fueron cumplidos. Revisar configuración.")
print("="*70)

📊 RESUMEN DE MÉTRICAS DE RENDIMIENTO - PROYECTO DATA PIPELINE

🖥️ Entorno de Ejecución:
   - Plataforma: Docker Container
   - Python: 3.11.6
   - Fecha: 2026-02-09 21:34:13

----------------------------------------------------------------------

📈 TABLA DE TIEMPOS (Copiar al informe):

| Operación                  | Tiempo Real | Objetivo    | Cumple |
|:---------------------------|:-----------:|:-----------:|:------:|
| Ingesta Cassandra (100k)   |    13.17 s  | < 5 min     | ✅     |
| Transformación Spark ETL   |     2.09 s  | < 2 min     | ✅     |
| Consulta Top 10 (CH)       |   0.0278 s  | < 3 seg     | ✅     |
| Consulta Promedio (CH)     |   0.0079 s  | < 3 seg     | ✅     |

💡 CONCLUSIÓN:
   ✅ TODOS los objetivos de rendimiento fueron CUMPLIDOS.


## 7. Exportar Métricas Finales a Markdown

In [9]:
# Actualizar archivo de métricas con consultas
metricas['consultas_clickhouse'] = {
    'consulta_top10': round(tiempo_consulta_top10, 4),
    'consulta_promedio': round(tiempo_consulta_promedio, 4),
    'registros_analizados': total_registros,
    'timestamp': datetime.now().isoformat()
}

with open(METRICS_FILE, 'w') as f:
    json.dump(metricas, f, indent=2)

# Generar archivo Markdown para el informe (en el mismo directorio)
output_md = '../docs/metricas_reales.md'

with open(output_md, 'w', encoding='utf-8') as f:
    f.write("# 📊 Métricas de Rendimiento - Valores Reales\n\n")
    f.write(f"**Fecha de ejecución:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    # 1. Diagrama de Arquitectura
    f.write("## 1. Diagrama de Arquitectura de Datos\n\n")
    f.write("```mermaid\n")
    f.write("graph LR\n")
    f.write("    A[Fuente de Datos] -->|Generador Python| B(Cassandra OLTP)\n")
    f.write("    B -->|Ingesta Paralela| C{Apache Spark}\n")
    f.write("    C -->|Transformación ETL| D(ClickHouse OLAP)\n")
    f.write("    D -->|Consultas SQL| E[Reporte Analítico]\n")
    f.write("    style B fill:#1f77b4,stroke:#333,stroke-width:2px,color:white\n")
    f.write("    style C fill:#d62728,stroke:#333,stroke-width:2px,color:white\n")
    f.write("    style D fill:#ff7f0e,stroke:#333,stroke-width:2px,color:white\n")
    f.write("```\n\n")
    
    # 2. Tabla Comparativa
    f.write("## 2. Tabla Comparativa de Rendimiento\n\n")
    f.write("| Operación | Tiempo Real | Objetivo | Cumple |\n")
    f.write("|:---|:---:|:---:|:---:|\n")
    f.write(f"| Ingesta Cassandra (100k) | {tiempo_ingesta:.2f} s | < 5 min | {'✅' if tiempo_ingesta < 300 else '❌'} |\n")
    f.write(f"| Transformación Spark ETL | {tiempo_etl:.2f} s | < 2 min | {'✅' if tiempo_etl < 120 else '❌'} |\n")
    f.write(f"| Consulta Top 10 (ClickHouse) | {tiempo_consulta_top10:.4f} s | < 3 seg | {'✅' if tiempo_consulta_top10 < 3 else '❌'} |\n")
    f.write(f"| Consulta Promedio (ClickHouse) | {tiempo_consulta_promedio:.4f} s | < 3 seg | {'✅' if tiempo_consulta_promedio < 3 else '❌'} |\n")
    
    # 3. Detalles
    f.write("\n## 3. Detalles de Ejecución\n\n")
    f.write(f"- **Registros insertados en Cassandra:** {metricas.get('ingesta_cassandra', {}).get('registros', 'N/A'):,}\n")
    f.write(f"- **Registros procesados por Spark:** {metricas.get('etl_spark', {}).get('registros_entrada', 'N/A'):,}\n")
    f.write(f"- **Registros en ClickHouse:** {total_registros:,}\n")
    
    # 4. Análisis Comparativo
    f.write("\n## 4. Análisis Comparativo: Cassandra vs ClickHouse\n\n")
    f.write("### ¿Por qué Cassandra para la Ingesta (OLTP)?\n")
    f.write("- **Escritura Optimizada:** Su arquitectura *Log-Structured Merge Tree* permite escrituras masivas secuenciales extremadamente rápidas.\n")
    f.write("- **Disponibilidad:** Su diseño *masterless* garantiza que el sistema siempre acepte escrituras, ideal para la captura de datos en tiempo real.\n")
    f.write("- **Escalabilidad Lineal:** Permite agregar nodos para aumentar la capacidad de escritura sin tiempos de inactividad.\n\n")
    f.write("### ¿Por qué ClickHouse para Analítica (OLAP)?\n")
    f.write("- **Almacenamiento Columnar:** Lee solo las columnas necesarias para la consulta (ej. `monto_total`), ignorando el resto, lo que acelera dramáticamente las agregaciones.\n")
    f.write("- **Compresión de Datos:** Almacena columnas de tipos similares juntas, logrando tasas de compresión altas y reduciendo E/S de disco.\n")
    f.write("- **Motores de Agregación:** Utiliza instrucciones vectoriales (SIMD) para procesar millones de filas en milisegundos, como se evidencia en los tiempos de consulta (< 0.02s).\n")

print(f"✅ Métricas exportadas a:")
print(f"   - JSON: {METRICS_FILE}")
print(f"   - Markdown: {output_md}")
print(f"\n📝 El informe se ha generado automáticamente en {output_md}.")

✅ Métricas exportadas a:
   - JSON: ../docs/metricas.json
   - Markdown: ../docs/metricas_reales.md

📝 El informe se ha generado automáticamente en ../docs/metricas_reales.md.
